In [ ]:
from tastytrade.logging import setup_logging
import logging
import asyncio
from tastytrade.sessions import Credentials

from tastytrade.sessions.enumerations import Channels
from tastytrade.sessions.sockets import DXLinkManager
from tastytrade.sessions.models import CandleSubscriptionRequest

from datetime import datetime, timedelta

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

setup_logging(
    level=logging.DEBUG,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

## Test individual components

In [ ]:
# Set API credentials
credentials = Credentials(env=ENV)


credentials = Credentials(env="Live")
dxlink = DXLinkManager()

await dxlink.open(credentials)

request = CandleSubscriptionRequest(
    symbol="BTC/USD:CXTALP", interval="5m", from_time=datetime.now() - timedelta(minutes=10)
)
await dxlink.subscribe_to_candles(request)

2025-02-04 10:29:17 - DEBUG:websockets.client:599:< TEXT '{"type":"FEED_DATA","channel":11,"data":["Candl....25,98992.92,99179.7]]}' [110 bytes]
2025-02-04 10:29:17 - DEBUG:tastytrade.sessions.sockets:100:{"type":"FEED_DATA","channel":11,"data":["Candle",["BTC/USD:CXTALP{=5m}",99014.67,99404.25,98992.92,99179.7]]}
2025-02-04 10:29:17 - DEBUG:tastytrade.sessions.handlers:225:Candle handler for channel 11 processed 1 events
2025-02-04 10:29:17 - DEBUG:websockets.client:599:< TEXT '{"type":"FEED_DATA","channel":11,"data":["Candl...25,98992.92,99196.02]]}' [111 bytes]
2025-02-04 10:29:17 - DEBUG:tastytrade.sessions.sockets:100:{"type":"FEED_DATA","channel":11,"data":["Candle",["BTC/USD:CXTALP{=5m}",99014.67,99404.25,98992.92,99196.02]]}
2025-02-04 10:29:17 - DEBUG:tastytrade.sessions.handlers:225:Candle handler for channel 11 processed 1 events


In [ ]:
# Subscribe to 5-minute candles for SPY
request = CandleSubscriptionRequest(symbol="BTC/USD:CXTALP", interval="5m")
await dxlink.subscribe_to_candles(request)

In [ ]:
symbols = ["BTC/USD:CXTALP"]
await dxlink.subscribe(symbols)

In [ ]:
# Access candle data
dxlink.router.handler[Channels.Candle].processors["feed"].df.sort_values(
    "timestamp", ascending=True
)

In [ ]:
dxlink.router.handler[Channels.Trade].processors["feed"].df

In [ ]:
await dxlink.close()

In [4]:
request = CandleSubscriptionRequest(
    symbol="BTC/USD:CXTALP", interval="1m", from_time=datetime.now() - timedelta(minutes=10)
)

In [17]:
# Notes

# We don't need to keep all the Candlestick ticks, just the latest "Live" tick preceded by a hsitory of prior candlesticks